In [ ]:
import datatable as dt
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch
from torch import nn
from torch.nn import functional as F
from torch.optim.lr_scheduler import _LRScheduler
import janestreet
from tqdm import tqdm
import pandas as pd
import os

In [ ]:
data = dt.fread('/kaggle/input/jane-street-market-prediction/train.csv') #用datarable可以加速读取csv文件
data = data.to_pandas()

In [ ]:
"""
Reduce Memory Usage by 50%
https://www.kaggle.com/tomwarrens/nan-values-depending-on-time-of-day
"""

## Reduce Memory

def reduce_memory_usage(df):
    
    start_memory = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe is {start_memory} MB")
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != 'object':
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            
            else:
#                 reducing float16 for calculating numpy.nanmean
#                 if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
#                     df[col] = df[col].astype(np.float16)
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    pass
        else:
            df[col] = df[col].astype('category')
    
    end_memory = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe after reduction {end_memory} MB")
    print(f"Reduced by {100 * (start_memory - end_memory) / start_memory} % ")
    return df

data = reduce_memory_usage(data)

In [ ]:
data = data.drop(data[(data['weight'] == 0) | (data['date'] <= 85)].index) #删除前85天的数据和weight等于0的数据
ignore_columns = ['resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp','ts_id','date']
features = [col for col in data.columns if col not in ignore_columns]

In [ ]:
data = data.fillna(data.mean()) #平均值填充
data['action'] = (data['resp'] > 0).astype('int') #resp大于0使action取1，也就是买入
data = data.drop(columns=ignore_columns)

In [ ]:
print(torch.cuda.is_available())

In [ ]:
class Timeseries_Dataset(Dataset):
    """
    Custom Dataset subclass.
    Serves as input to DataLoader to transform X
      into sequence data using rolling window.
    DataLoader using this dataset will output batches
      of `(batch_size, seq_len, n_features)` shape.
    Suitable as an input to RNNs.
    """

    def __init__(self, X: np.ndarray, y: np.ndarray, seq_len: int = 32):
        self.X = torch.tensor(X).float()
        self.y = torch.tensor(y).float()
        self.seq_len = seq_len

    def __len__(self):
        return self.X.__len__() - (self.seq_len - 1)

    def __getitem__(self, index):
        return {'x': torch.tensor(self.X[index:index + self.seq_len], dtype=torch.float),
                'y': torch.tensor(self.y[index + self.seq_len - 1], dtype=torch.long)}
class LSTMClassifier(nn.Module):
    """Very simple implementation of LSTM-based time-series classifier."""
    
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.rnn = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        h0, c0 = self.init_hidden(x)
        out, (hn, cn) = self.rnn(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out
    
    def init_hidden(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        return [t.to(device) for t in (h0, c0)]
        

In [ ]:
num_epochs = 10
batch_size = 4096
lr = 0.0005
input_dim = 131
hidden_dim = 256
layer_dim = 2
output_dim = 2
seq_dim = 32
target_column = 'action'

feature_columns = data.columns[~data.columns.isin([target_column])]
train, validation = data[:int(len(data) * 0.8)], data[int(len(data) * 0.2):]  #pandas数据前80%作训练，后20%作预测
train_features, train_target = train[feature_columns], train[[target_column]]
validation_features, validation_target = validation[feature_columns], validation[[target_column]]
train_dataset = Timeseries_Dataset(X=train_features.values, y=train_target.values, seq_len=seq_dim) #转成numpy数据
validation_dataset = Timeseries_Dataset(X=validation_features.values, y=validation_target.values, seq_len=seq_dim)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
weight = '/kaggle/input/weight-lstm/best_30.pth' #???

phase_training = True
if os.path.exists(weight):
    phase_training = False

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = LSTMClassifier(input_dim, hidden_dim, layer_dim, output_dim)
model = model.to(device)
if phase_training:
    iterations_per_epoch = len(train_loader)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr=lr)
    print('Start model training ...')
    best_acc = 0.0
    patience, trials = 100, 0
    for epoch in range(1, num_epochs + 1):
        for i, train_batch in enumerate(train_loader):
            model.train()
            features = train_batch['x'].to(device)
            targets = train_batch['y'].to(device)
            targets = torch.squeeze(targets)
            preds = model(features)
            loss = criterion(preds, targets) #？？？ok
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch} best model saved with loss: {loss:2.2}')

        model.eval()
        correct, total = 0, 0
        for valid_batch in validation_loader:
            features = valid_batch['x'].to(device)
            targets = valid_batch['y'].to(device)
            targets = torch.squeeze(targets)
            preds = model(features)
            preds = F.log_softmax(preds, dim=1).argmax(dim=1) #输出结果一行两个特征【0,1】，取出较大的特征对应的列数0或者1作为结果
            total += targets.size(0)
            correct += (preds == targets).sum().item() #tensor类型转换成python数字

        acc = correct / total

        if epoch % 1 == 0:
            print(f'Epoch: {epoch:3d}. Loss: {loss.item():.4f}. Acc.: {acc:2.2%}')

        if acc > best_acc:
            trials = 0
            best_acc = acc
            torch.save(model.state_dict(), 'best.pth') #？？？ok
            print(f'Epoch {epoch} best model saved with accuracy: {best_acc:2.2%}')
        else:
            trials += 1
            if trials >= patience:
                print(f'Early stopping on epoch {epoch}')
                break
    print('Training Complete !!!')

In [ ]:
if phase_training: #???ok
    model.load_state_dict(torch.load('best.pth'))
else:
    model.load_state_dict(torch.load(weight))
    
model.eval()
X_test = None
env = janestreet.make_env()
env_iter = env.iter_test()
for (test_df, pred_df) in tqdm(env_iter):
    if test_df['weight'].item() > 0:
        test_df = pd.DataFrame(test_df, columns=feature_columns)
        test_df = test_df.fillna(test_df.mean())
        if X_test is None:
            X_test = np.concatenate([test_df for _ in range(seq_dim)],axis=0)
        X_test = np.concatenate([X_test[1:], test_df] ,axis=0)
        preds = model(torch.tensor(X_test[np.newaxis,:], dtype=torch.float).to(device))
        preds = F.log_softmax(preds, dim=1).argmax(dim=1)
        action = preds.cpu().detach().numpy()
        pred_df.action = action
    else:
        pred_df.action = 0
    env.predict(pred_df)